# Reconstruct and model glycolysis

Building and simulation pathways using COBRApy. 


In [1]:
import cobra

from fba4l_course_material.resources.glycolysis_pathway import metabolites_and_info, reactions_and_info

In [2]:
# The reactions and metabolites to be added in this exercise
try:
    import escher
    builder = escher.Builder(map_json='../resources/glycolysis_map.json')
    builder
except:
    pass

---------
## A) Add the pathway metabolites and their information to a new model

The goal of this exercise will be to build a simulate the glycolysis synthesis pathway shown above. 

This section will introduce the first steps of adding the pathway to model.

### 1) Initialize an empty COBRApy model

In [3]:
# Initiate empty model
model = cobra.Model()

Print the number of metabolites and reactions in the model

In [4]:
# with no reactions
print('Number of reactions: ' + str(len(model.reactions)))
print('Number of metabolites: ' + str(len(model.metabolites)))

Number of reactions: 0
Number of metabolites: 0


### 2) Add metabolites and info to the model

**Hints:**

1) The information regarding the reactions and metabolites involved in this pathway can be found in `reactions_and_info` and `metabolites_and_info`, respectively. These are imported above.
  - These are imported in the form of a **dictionary**
  - Print the contents of `metabolites_and_info`

In [5]:
print(metabolites_and_info)

{'atp_c': {'name': 'ATP C10H12N5O13P3', 'formula': 'C10H12N5O13P3', 'charge': -4, 'compartment': 'c'}, 'glc__D_c': {'name': 'D-Glucose', 'formula': 'C6H12O6', 'charge': 0, 'compartment': 'c'}, 'adp_c': {'name': 'ADP C10H12N5O10P2', 'formula': 'C10H12N5O10P2', 'charge': -3, 'compartment': 'c'}, 'g6p_c': {'name': 'D-Glucose 6-phosphate', 'formula': 'C6H11O9P', 'charge': -2, 'compartment': 'c'}, 'h_c': {'name': 'H+', 'formula': 'H', 'charge': 1, 'compartment': 'c'}, 'f6p_c': {'name': 'D-Fructose 6-phosphate', 'formula': 'C6H11O9P', 'charge': -2, 'compartment': 'c'}, 'fdp_c': {'name': 'D-Fructose 1,6-bisphosphate', 'formula': 'C6H10O12P2', 'charge': -4, 'compartment': 'c'}, 'dhap_c': {'name': 'Dihydroxyacetone phosphate', 'formula': 'C3H5O6P', 'charge': -2, 'compartment': 'c'}, 'g3p_c': {'name': 'Glyceraldehyde 3-phosphate', 'formula': 'C3H5O6P', 'charge': -2, 'compartment': 'c'}, 'nad_c': {'name': 'Nicotinamide adenine dinucleotide', 'formula': 'C21H26N7O14P2', 'charge': -1, 'compartment'

----

- A dictionary is a Python data structure that consists of both `keys` and `values`. This allows you to look up quantities based on the `key` you are interested in. An example of this is shown below for ATP

----

In [6]:
# Look up the information assocated with the histidine metabolite (his__L_c)
print('Values associated with atp_c = ', metabolites_and_info['atp_c'])
# This returns another dictionary where the formula, etc. can be looked up with:
print('Formula = ', metabolites_and_info['atp_c']['formula'])

Values associated with atp_c =  {'name': 'ATP C10H12N5O13P3', 'formula': 'C10H12N5O13P3', 'charge': -4, 'compartment': 'c'}
Formula =  C10H12N5O13P3


2) To leverage dictionary properties to add the metabolites to model more efficiently, iterate through all of the dictionary entries using the `items` method and print the `keys` and `values`

In [7]:
for key, value in metabolites_and_info.items():
    print(key, value)

atp_c {'name': 'ATP C10H12N5O13P3', 'formula': 'C10H12N5O13P3', 'charge': -4, 'compartment': 'c'}
glc__D_c {'name': 'D-Glucose', 'formula': 'C6H12O6', 'charge': 0, 'compartment': 'c'}
adp_c {'name': 'ADP C10H12N5O10P2', 'formula': 'C10H12N5O10P2', 'charge': -3, 'compartment': 'c'}
g6p_c {'name': 'D-Glucose 6-phosphate', 'formula': 'C6H11O9P', 'charge': -2, 'compartment': 'c'}
h_c {'name': 'H+', 'formula': 'H', 'charge': 1, 'compartment': 'c'}
f6p_c {'name': 'D-Fructose 6-phosphate', 'formula': 'C6H11O9P', 'charge': -2, 'compartment': 'c'}
fdp_c {'name': 'D-Fructose 1,6-bisphosphate', 'formula': 'C6H10O12P2', 'charge': -4, 'compartment': 'c'}
dhap_c {'name': 'Dihydroxyacetone phosphate', 'formula': 'C3H5O6P', 'charge': -2, 'compartment': 'c'}
g3p_c {'name': 'Glyceraldehyde 3-phosphate', 'formula': 'C3H5O6P', 'charge': -2, 'compartment': 'c'}
nad_c {'name': 'Nicotinamide adenine dinucleotide', 'formula': 'C21H26N7O14P2', 'charge': -1, 'compartment': 'c'}
pi_c {'name': 'Phosphate', 'formu

3) Add the metabolites, along with the metabolite information to the empty model.
 - Then print the number of metabolites in the model 

In [8]:
# Add all the remaining metabolites involved in the pathway
for met_id, info in metabolites_and_info.items():
    met = cobra.Metabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    model.add_metabolites(met)

In [9]:
print('Number of metabolites: ' + str(len(model.metabolites)))

Number of metabolites: 18


-------
## B) Add reactions to model and simulate

### 1) Like above, add the reaction info found in `reactions_and_info` to the model
 

In [10]:
for reaction, info in reactions_and_info.items():
    print(reaction)
    reaction_obj = cobra.Reaction(reaction)
    reaction_obj.lower_bound = info['lower_bound']
    reaction_obj.upper_bound = info['upper_bound']
    reaction_obj.name = info['name']
    model.add_reactions([reaction_obj])
    reaction_obj.add_metabolites(info['stoichiometry'])
    reaction_obj.gene_reaction_rule = info['gene_reaction_rule']

HEX1
PGI
PFK
TPI
FBA
GAPD
PGK
PGM
ENO
PYK


### 2)  Mass balance check each reaction added to the pathway using `reaction.check_mass_balance()` method

This is an important step in quality controlling any metabolic model. Failing to do so could result in incorrect reactions and incorrect predictions


In [11]:
for r in model.reactions:
    print(r.id,  r.check_mass_balance())

HEX1 {}
PGI {}
PFK {}
TPI {}
FBA {'charge': 2.0, 'C': -3.0, 'H': -5.0, 'O': -6.0, 'P': -1.0}
GAPD {}
PGK {}
PGM {}
ENO {}
PYK {}


### 3) If a reaction is not mass balanced, correct it with `reaction.add_metabolites()` and confirm it is now mass balance
The proper stoichiometry for the reaction can be found at [ecocyc](http://www.ecocyc.com) or [bigg](http://bigg.ucsd.edu). Databases like these are useful references for metabolic reconstructions and understanding models.

**Hint:**
1) Metabolites can be add and removed from reactions using `add_metabolites` on an existing reaction like below

In [12]:
model.reactions.GAPD.add_metabolites({'atp_c': 1.0}, combine=False)
print('With atp: ', model.reactions.GAPD.reaction)
model.reactions.GAPD.add_metabolites({'atp_c': 0}, combine=False)
print('Back to original reaction: ', model.reactions.GAPD.reaction)

With atp:  g3p_c + nad_c + pi_c <=> 13dpg_c + atp_c + h_c + nadh_c
Back to original reaction:  g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c


In [13]:
model.reactions.FBA.add_metabolites({'dhap_c': 1.0})
print(model.reactions.FBA.check_mass_balance())

{}


### 4) Add boundary reactions to act as source and sink for glucose and pyruvate

Models require inputs and outputs in order to simulate. In other words, flux going into a metabolite must ultimately go somewhere.

The reactions should have the following info:

| ID      | Reaction       |   Lower Bound | Upper Bound 
| :-------------: |:-------------:|:-------------:|:-------------:|
| EX_glc__D_c | glc__D_c <=>     | -1 | 1000 |
| EX_pyr_c | pyr_c -->     | 0 | 1000 |
| EX_h2o_c | h2o_c <=>     | -1000 | 1000 |
| EX_h_c | h_c <=>     | -1000 | 1000 |
| EX_pi_c | pi_c <=>     | -1000 | 1000 |

These reactions are called **Exchange Reactions** and by convention begin with 'EX_'. Further, by convention a negative lower bound/flux corresponds to a model input (or metabolite uptake from the growth media/growth condition). Therefore, a lower bound of 1 means that the model can take up, at most, 1 $\frac{mmol}{gDW \cdot hr}$ of glucose

In [14]:
for met in ['glc__D_c', 'pyr_c', 'h2o_c', 'h_c', 'pi_c']:
    rxn = cobra.Reaction('EX_%s' % met)
    model.add_reactions([rxn])
    rxn.add_metabolites({met: -1})
    if met == 'glc__D_c':
        rxn.lower_bound = -1 # by convention negative exchange flux = uptake
    elif met == 'pyr_c':
        rxn.lower_bound = 0
    else:
        rxn.lower_bound = -1000
        


### 5) View the growth media of the model based on the exchange reactions using `model.medium`

In [15]:
model.medium

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


{'EX_glc__D_c': 1, 'EX_h2o_c': 1000, 'EX_h_c': 1000, 'EX_pi_c': 1000}

### 6) Set the model objective to pyruvate production ('EX_pyr_e') and optimize

The objective corresponds to the reaction flux that the optimization will either maximize or minimize (by default maximize).

In [16]:
model.objective = 'EX_pyr_c'
print(model.objective)

Maximize
1.0*EX_pyr_c - 1.0*EX_pyr_c_reverse_5a272


In [17]:
model.objective = 'EX_pyr_c'
model.optimize()

,fluxes,reduced_costs
HEX1,0.0,0.0
PGI,0.0,0.0
PFK,0.0,0.0
TPI,0.0,0.0
FBA,0.0,0.0
...,...,...
EX_glc__D_c,0.0,0.0
EX_pyr_c,0.0,0.0
EX_h2o_c,0.0,0.0
EX_h_c,0.0,0.0


**Still no flux through the reaction?  Why?**

Flux balance analysis assumes that the system is opperating at steady state (meaning the concentration of the metabolites in the system do not increase or decrease over time). As a consequence of this, each metabolite that participates in the pathway must be created and consumed at the same rate. 


ATP and NADH are generated in glycolysis but not used
- Add ATPM and DM_nadh reactions from bigg

In [18]:
# add ATPM and NADHM
atpm = {'atp_c': -1, 'h2o_c': -1, 'adp_c': 1, 'h_c': 1, 'pi_c': 1}
rxn = cobra.Reaction('ATPM')
model.add_reactions([rxn])
rxn.add_metabolites(atpm)

nadhm = {'nadh_c': -1, 'h_c': 1, 'nad_c': 1}
rxn = cobra.Reaction('DM_nadh')
model.add_reactions([rxn])
rxn.add_metabolites(nadhm)

In [19]:
model.optimize()

,fluxes,reduced_costs
HEX1,1.0,0.0
PGI,1.0,0.0
PFK,1.0,0.0
TPI,1.0,0.0
FBA,1.0,0.0
...,...,...
EX_h2o_c,0.0,0.0
EX_h_c,6.0,0.0
EX_pi_c,0.0,0.0
ATPM,2.0,0.0


### 7) Summarzing the metabolic fate of pyruvate / NADH
Pyruvate and NADH facillitate metabolic processes in pathways we will model later.

For now, summarize the metabolic fates of these two metabolites with the following reactions (after adding a reaction to the model, you can use the reactions `build_reaction_from_sring` method):

- **nadh_to_atp**: nadh_c+ .5 o2_c + 3.5 h_c + 2.5 pi_c + 2.5 adp_c -> nad_c + 3.5 h2o_c + 2.5 atp_c
  - 2.5 ATP per NADH (electron transport chain)
- **pyr_to_atp**: pyr_c + 3 h2o_c + 12.5 h_c + 12.5 pi_c + 12.5 adp_c -> 3 co2_c + 12.5 h2o_c + 12.5 atp_c + 9 h_c
  - 15 ATP per pyruvate (TCA cycle + electron transport chain)
  
  
**Note**: also need to remove the DM_nadh reaction
- Can use reaction's `remove_from_model` method 

In [20]:
r = cobra.Reaction('NADH_to_ATP')
model.add_reactions([r])
r.build_reaction_from_string('nadh_c+ .5 o2_c + 3.5 h_c + 2.5 pi_c + 2.5 adp_c -> nad_c + 3.5 h2o_c + 2.5 atp_c')

r = cobra.Reaction('pyr_to_ATP')
model.add_reactions([r])
r.build_reaction_from_string('pyr_c + 3 h2o_c + 12.5 h_c + 12.5 pi_c + 12.5 adp_c -> 3 co2_c + 12.5 h2o_c + 12.5 atp_c + 9 h_c')


model.reactions.DM_nadh.remove_from_model()

unknown metabolite 'o2_c' created
unknown metabolite 'co2_c' created


#### Note that you need to add `o2_c` and `co2_c` metabolites into the model
These therefore need exchange reactions into and out of the system
- Use upper and lower bound of 1000 and -1000, respectively

In [21]:
for met in ['o2_c', 'co2_c']:
    rxn = cobra.Reaction('EX_%s' % met)
    model.add_reactions([rxn])
    rxn.add_metabolites({met: -1})
    rxn.lower_bound = -1000

#### Set objective to `ATPM` and optimize

In [22]:
model.objective = 'ATPM'
model.optimize()

,fluxes,reduced_costs
HEX1,1.0,0.000000e+00
PGI,1.0,0.000000e+00
PFK,1.0,0.000000e+00
TPI,1.0,0.000000e+00
FBA,1.0,0.000000e+00
...,...,...
ATPM,32.0,0.000000e+00
NADH_to_ATP,2.0,0.000000e+00
pyr_to_ATP,2.0,-1.776357e-15
EX_o2_c,-1.0,0.000000e+00


### 8) Modeling the oxygen dependance of ATP production

1. Set objective to ATPM
2. Set the lower bound of `EX_o2_c` to 50 values ranging from -2 to 0
  - Can use numpy's `linspace` function
3. Optimize and print the ATPM values

In [23]:
import numpy as np
for lb in np.linspace(-2, 0, 50):
    model.reactions.EX_o2_c.lower_bound = lb
    print('ATP yield when oxygen uptake is %.2f = %.2f' % (lb, model.optimize().objective_value))

ATP yield when oxygen uptake is -2.00 = 32.00
ATP yield when oxygen uptake is -1.96 = 32.00
ATP yield when oxygen uptake is -1.92 = 32.00
ATP yield when oxygen uptake is -1.88 = 32.00
ATP yield when oxygen uptake is -1.84 = 32.00
ATP yield when oxygen uptake is -1.80 = 32.00
ATP yield when oxygen uptake is -1.76 = 32.00
ATP yield when oxygen uptake is -1.71 = 32.00
ATP yield when oxygen uptake is -1.67 = 32.00
ATP yield when oxygen uptake is -1.63 = 32.00
ATP yield when oxygen uptake is -1.59 = 32.00
ATP yield when oxygen uptake is -1.55 = 32.00
ATP yield when oxygen uptake is -1.51 = 32.00
ATP yield when oxygen uptake is -1.47 = 32.00
ATP yield when oxygen uptake is -1.43 = 32.00
ATP yield when oxygen uptake is -1.39 = 32.00
ATP yield when oxygen uptake is -1.35 = 32.00
ATP yield when oxygen uptake is -1.31 = 32.00
ATP yield when oxygen uptake is -1.27 = 32.00
ATP yield when oxygen uptake is -1.22 = 32.00
ATP yield when oxygen uptake is -1.18 = 32.00
ATP yield when oxygen uptake is -1